In [ ]:
from sympy import *
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
init_printing()
print("imports")

imports


In [ ]:
#Constants
grav = 9.81
mass = 1.552    # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.04 # The z distance from the quadrotor frame to the mocap markers in meters
print("constants")

constants


In [ ]:
# Define symbols
px_inW, py_inW, pz_inW = symbols('p_x, p_y, p_z')
vx_inB, vy_inB, vz_inB = symbols('v_x, v_y, v_z')
phi, theta, psi = symbols('phi, theta, psi')
wx_inB, wy_inB, wz_inB = symbols('omega_x, omega_y, omega_z')
taux_inB, tauy_inB, tauz_inB, fz_inB = symbols('tau_x, tau_y, tau_z, f_z')
print("symbols")

symbols


In [ ]:
p_inW = Matrix([[px_inW],[py_inW], [pz_inW]])
v_inB = Matrix([[vx_inB],[vy_inB],[vz_inB]])
w_inB = Matrix([[wx_inB],[wy_inB], [wz_inB]])
Rx = Matrix([[1,0,0],[0, cos(phi), -sin(phi)],[0, sin(phi),  cos(phi)]])
Ry = Matrix([[ cos(theta), 0, sin(theta)],[0, 1,0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0],[sin(psi),  cos(psi), 0],[0,0,1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T
pprint(R_ofW_inB)

⎡           cos(ψ)⋅cos(θ)                          sin(ψ)⋅cos(θ)               ↪
⎢                                                                              ↪
⎢sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)  sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)    ↪
⎢                                                                              ↪
⎣sin(φ)⋅sin(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(ψ) + sin(ψ)⋅sin(θ)⋅cos(φ)   ↪

↪    -sin(θ)   ⎤
↪              ⎥
↪ sin(φ)⋅cos(θ)⎥
↪              ⎥
↪ cos(φ)⋅cos(θ)⎦


In [ ]:
tau_inB = Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = simplify(tau_inB)
grav_inW = Matrix([[0.],
                       [0.],
                       [-mass * grav]])
grav_inB = R_ofW_inB @ grav_inW
f_inB = grav_inB + Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = simplify(f_inB)
pprint(f_inB)

⎡      15.22512⋅sin(θ)       ⎤
⎢                            ⎥
⎢  -15.22512⋅sin(φ)⋅cos(θ)   ⎥
⎢                            ⎥
⎣f_z - 15.22512⋅cos(φ)⋅cos(θ)⎦


In [ ]:
Ixx = (1/12)*(mass/4)*lz**2
Iyy = Ixx
Izz = (1/12)*(mass/4)*lz**2*2
I_inB = Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])
pprint(I_inB)

⎡5.17333333333333e-5          0.0                  0.0         ⎤
⎢                                                              ⎥
⎢        0.0          5.17333333333333e-5          0.0         ⎥
⎢                                                              ⎥
⎣        0.0                  0.0          0.000103466666666667⎦


In [ ]:
v_inW = R_ofB_inW @ v_inB
xyz_dot = simplify(v_inW)
pprint(xyz_dot)

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s ↪
⎢                                                                              ↪
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c ↪
⎢                                                                              ↪
⎣                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ ↪

↪ in(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ))⎤
↪                              ⎥
↪ os(ψ) - sin(ψ)⋅sin(θ)⋅cos(φ))⎥
↪                              ⎥
↪ )                            ⎦


In [ ]:
v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB))
v_inB_dot = simplify(v_inB_dot)
pprint(v_inB_dot)

⎡              -1.0⋅ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)              ⎤
⎢                                                                    ⎥
⎢            1.0⋅ωₓ⋅v_z - 1.0⋅ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)            ⎥
⎢                                                                    ⎥
⎣0.644329896907216⋅f_z - 1.0⋅ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos(φ)⋅cos(θ)⎦


In [ ]:
ex = Matrix([[1], [0], [0]])
ey = Matrix([[0], [1], [0]])
ez = Matrix([[0], [0], [1]])
M = Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)
M_inv = simplify(M.inv())
rpy_dot = simplify(M_inv@w_inB)
pprint(rpy_dot)

⎡ωₓ + ω_y⋅sin(φ)⋅tan(θ) + ω_z⋅cos(φ)⋅tan(θ)⎤
⎢                                          ⎥
⎢         ω_y⋅cos(φ) - ω_z⋅sin(φ)          ⎥
⎢                                          ⎥
⎢         ω_y⋅sin(φ) + ω_z⋅cos(φ)          ⎥
⎢         ───────────────────────          ⎥
⎣                 cos(θ)                   ⎦


In [ ]:
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = simplify(w_inB_dot)
pprint(w_inB_dot)

⎡-1.0⋅ω_y⋅ω_z + 19329.8969072165⋅τₓ⎤
⎢                                  ⎥
⎢1.0⋅ωₓ⋅ω_z + 19329.8969072165⋅τ_y ⎥
⎢                                  ⎥
⎣       9664.94845360825⋅τ_z       ⎦


In [ ]:
f = Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)
pprint(f)

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s ↪
⎢                                                                              ↪
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c ↪
⎢                                                                              ↪
⎢                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ ↪
⎢                                                                              ↪
⎢                                 -1.0⋅ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)     ↪
⎢                                                                              ↪
⎢                               1.0⋅ωₓ⋅v_z - 1.0⋅ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)   ↪
⎢                                                                              ↪
⎢                   0.644329896907216⋅f_z - 1.0⋅ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos ↪
⎢                                                                              ↪
⎢                           

In [ ]:
from scipy.integrate import solve_ivp
time = (0,120)
eval = np.linspace(time[0],time[1],1000)
phidot = w
initial = [0,0,1,
           0,0,0,
           0,0,0,
           0,0,0]
eom = [xyz_dot[0],xyz_dot[1],xyz_dot[2],]
sol = solve_ivp(eom, time, initial, t_eval = eval, method = 'RK45')
x,y,z,phi,theta,psi = sol.y[:6]

Traceback (most recent call last):
  File "c:\Users\emlo2\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 18, in <module>
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_ivp\ivp.py", line 621, in solve_ivp
    solver = method(fun, t0, y0, tf, vectorized=vectorized, **options)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_ivp\rk.py", line 94, in __init__
    self.f = self.fun(self.t, self.y)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_ivp\base.py", line 154, in fun
    return self.fun_single(t, y)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C

In [ ]:

t = np.linspace(0, 10, 100)  # Time from 0 to 10 seconds
m = 1

x =   # x-component (linear motion)
y = np.sin(t)  # y-component (sinusoidal motion)
z = np.cos(t)  # z-component (cosine motion)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(x, y, z, label='Trajectory (Position)')
ax.set_title('Trajectory vs Time')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Position')
ax.legend()
ax.grid()

# # Plot Velocity
# plt.figure()
# plt.plot(t, v_inB_dot, label='Velocity', color='orange')
# plt.title('Velocity vs Time')
# plt.xlabel('Time (s)')
# plt.ylabel('Velocity')
# plt.legend()
# plt.grid()

# # Plot Acceleration
# plt.figure()
# plt.plot(t, w_inB_dot, label='Acceleration', color='red')
# plt.title('Acceleration vs Time')
# plt.xlabel('Time (s)')
# plt.ylabel('Acceleration')
# plt.legend()
# plt.grid()

plt.show()


Traceback (most recent call last):
  File "c:\Users\emlo2\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 22, in <module>
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\pyplot.py", line 3794, in plot
    return gca().plot(
           ^^^^^^^^^^^
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\axes\_axes.py", line 1779, in plot
    lines = [*self._get_lines(self, *args, data=data, **kwargs)]
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\axes\_base.py", line 296, in __call__
    yield from self._plot_args(
               ^^^^^^^^^^^^^^^^
  File "C:\Users\emlo2\AppData\Local\Programs\Python\Python312\Lib\